In [11]:
!pip install pyclustering


In [24]:
import pandas as pd

# moon dataset
dataset_names = ['circles0.3.csv', 'halfkernel.csv', 'moons1.csv', 'spiral1.csv', 'twogaussians33.csv', 'twogaussians42.ipynb']
datasets_ = []

for _, i in enumerate(dataset_names):
    print(i)
    dataset = pd.read_csv(str(i))
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    datasets_.append((i, X, y))

circles0.3.csv
halfkernel.csv
moons1.csv
spiral1.csv
twogaussians33.csv
twogaussians42.ipynb


ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


In [25]:
len(datasets_)

5

In [35]:
print(datasets_[0][0])

circles0.3.csv


In [13]:
from sklearn.metrics import accuracy_score
import numpy as np

def purity_score(y_true, y_pred):
    """Purity score

    To compute purity, each cluster is assigned to the class which is most frequent 
    in the cluster [1], and then the accuracy of this assignment is measured by counting 
    the number of correctly assigned documents and dividing by the number of documents.
    We suppose here that the ground truth labels are integers, the same with the predicted clusters i.e
    the clusters index.

    Args:
        y_true(np.ndarray): n*1 matrix Ground truth labels
        y_pred(np.ndarray): n*1 matrix Predicted clusters
    
    Returns:
        float: Purity score
    
    References:
        [1] https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-clustering-1.html
    """
    # matrix which will hold the majority-voted labels
    y_voted_labels = np.zeros(y_true.shape)
    # Ordering labels
    ## Labels might be missing e.g with set like 0,2 where 1 is missing
    ## First find the unique labels, then map the labels to an ordered set
    ## 0,2 should become 0,1
    labels = np.unique(y_true)
    ordered_labels = np.arange(labels.shape[0])
    for k in range(labels.shape[0]):
        y_true[y_true==labels[k]] = ordered_labels[k]
    # Update unique labels
    labels = np.unique(y_true)
    # We set the number of bins to be n_classes+2 so that 
    # we count the actual occurence of classes between two consecutive bin
    # the bigger being excluded [bin_i, bin_i+1[
    bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

    for cluster in np.unique(y_pred):
        hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)
        # Find the most present label in the cluster
        winner = np.argmax(hist)
        y_voted_labels[y_pred==cluster] = winner
    
    return accuracy_score(y_true, y_voted_labels)

In [4]:
# function defined to compute purity score using pyclustering for various distance measures


NameError: name 'distance_measures' is not defined

In [41]:
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.cluster.encoder import cluster_encoder
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import distance_metric


# define dictionary for distance measures
distance_measures = {'euclidean': 0, 'squared euclidean': 1, 'manhattan': 2, 'chebyshev': 3, 
                    'canberra': 5, 'chi-square': 6}

for _, dataset_data in enumerate(datasets_):
    X = dataset_data[1]
    y = dataset_data[2]
    # function defined to compute purity score using pyclustering for various distance measures
    def pyPurity(dist_measure):
        initial_centers = random_center_initializer(X, 3, random_state=5).initialize()
        # instance created for respective distance metric
        instanceKm = kmeans(X, initial_centers=initial_centers, metric=distance_metric(dist_measure))
        # perform cluster analysis
        instanceKm.process()
        # cluster analysis results - clusters and centers
        pyClusters = instanceKm.get_clusters()
        pyCenters = instanceKm.get_centers()
        # enumerate encoding type to index labeling to get labels
        pyEncoding = instanceKm.get_cluster_encoding()
        pyEncoder = cluster_encoder(pyEncoding, pyClusters, X)
        pyLabels = pyEncoder.set_encoding(0).get_clusters()
        # function purity score is defined in previous section
        return purity_score(y, pyLabels)

    # print results
    print('DATASET: ', dataset_data[0])
    for measure, value in distance_measures.items():
        print(f"The purity score for {measure} distance is {round(pyPurity(value)*100, 2)}%")
 

DATASET:  circles0.3.csv
The purity score for euclidean distance is 51.2%
The purity score for squared euclidean distance is 51.7%
The purity score for manhattan distance is 82.4%
The purity score for chebyshev distance is 52.7%
The purity score for canberra distance is 50.9%
The purity score for chi-square distance is 50.2%
DATASET:  halfkernel.csv
The purity score for euclidean distance is 59.6%
The purity score for squared euclidean distance is 59.6%
The purity score for manhattan distance is 58.8%
The purity score for chebyshev distance is 61.3%
The purity score for canberra distance is 63.3%
The purity score for chi-square distance is 63.4%
DATASET:  moons1.csv
The purity score for euclidean distance is 77.8%
The purity score for squared euclidean distance is 77.8%
The purity score for manhattan distance is 77.9%
The purity score for chebyshev distance is 76.4%
The purity score for canberra distance is 82.5%
The purity score for chi-square distance is 82.0%
DATASET:  spiral1.csv
T